In [282]:
import numpy as np
import pandas as pd

import random
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

from sklearn.preprocessing import StandardScaler
import pickle


In [283]:
dataset = pd.read_csv(r"npk_new.csv")
dataset

,N,P,K,LABEL
0,80,77,73,Beans
1,63,73,74,Beans
2,85,61,78,Beans
3,116,59,54,Beans
4,96,64,69,Beans
...,...,...,...,...
414,118,47,77,Watermelon
415,157,57,66,Watermelon
416,144,43,77,Watermelon
417,148,47,61,Watermelon


In [284]:
labels = dataset['LABEL'].unique().tolist()
labels.sort()
print(len(labels))
labels


17


['Beans',
 'Brinjal',
 'Cabbage',
 'Carrot',
 'Cauliflower',
 'Cowpea',
 'Cucumber',
 'Garden pea',
 'Garlic',
 'Lettuce',
 'Okra',
 'Onion',
 'Potato',
 'Snap bean',
 'Spinach',
 'Tomato',
 'Watermelon']

In [285]:
le = LabelEncoder()
le.fit(dataset['LABEL'])
dataset['LABEL']=le.transform(dataset['LABEL'])

In [286]:
dataset

,N,P,K,LABEL
0,80,77,73,0
1,63,73,74,0
2,85,61,78,0
3,116,59,54,0
4,96,64,69,0
...,...,...,...,...
414,118,47,77,16
415,157,57,66,16
416,144,43,77,16
417,148,47,61,16


In [287]:
# scaler = StandardScaler()
# #scaler.fit(X)
# dataset.iloc[0:,:-1] =scaler.fit_transform(dataset.iloc[0:,:-1]) 

In [288]:

# selecting the features and labels
X = dataset.iloc[:, :-1].values
Y = dataset.loc[:, ['LABEL']]

print(X)
print(Y)

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# splitting the model into training and testing set
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.25, random_state=100)


[[ 80  77  73]
 [ 63  73  74]
 [ 85  61  78]
 ...
 [144  43  77]
 [148  47  61]
 [109  58  61]]
     LABEL
0        0
1        0
2        0
3        0
4        0
..     ...
414     16
415     16
416     16
417     16
418     16

[419 rows x 1 columns]


In [289]:
result=['Beans',
 'Brinjal',
 'Cabbage',
 'Carrot',
 'Cauliflower',
 'Chilli',
 'Cowpea',
 'Cucumber',
 'Garden pea',
 'Garlic',
 'Lettuce',
 'Okra',
 'Onion',
 'Potato',
 'Snap bean',
 'Spinach',
 'Tomato',
 'Watermelon']

In [290]:

# # training a logistics regression model
# logmodel = LogisticRegression()
# logmodel.fit(X_train, y_train.values.ravel())
# predictions = logmodel.predict(X_test)
# print("***********************************************************")
# print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
# print("***********************************************************")

sgd_classifier = SGDClassifier(tol=-np.infty, loss='log')
sgd_classifier.fit(X_train, y_train)

predictions = sgd_classifier.predict(X_test)
print("***********************************************************")
print("Accuracy = " + str(accuracy_score(y_test.values.ravel(), predictions)))
print("***********************************************************")

C:\Users\PRAMILA\anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


***********************************************************
Accuracy = 0.21904761904761905
***********************************************************


In [291]:
test_data=[[100,500,50]]
o=sgd_classifier.predict(test_data)
print(o)
print(result[int(o[0])])

[2]
Cabbage


In [292]:
from sklearn.ensemble import RandomForestRegressor
 
 # create regressor object
regressor = RandomForestRegressor(n_estimators = 200, random_state = 32)
 
# fit the regressor with x and y data
regressor.fit(X_train, y_train.values.ravel())
predictions = regressor.predict(X_train)
print("***********************************************************")

print(regressor.score(X_train, y_train))

print("***********************************************************")

***********************************************************
0.8998393430910416
***********************************************************


In [293]:
pickle.dump(regressor, open('randomforest_model.pkl','wb'))

In [298]:
labels[11]

'Onion'

In [305]:
test_data=[[200,59,70]]
#print(regressor.predict([[]]))

# Loading model to compare the results
regressor = pickle.load(open('randomforest_model.pkl','rb'))
o=regressor.predict(test_data)
print(o)
print(labels[round(o[0])])


[9.655]
Okra


In [280]:
results = pd.DataFrame(sgd_classifier.predict_proba([[137,42,144]]).transpose())
results.rename(columns={0: "score"}, inplace=True)

results.reset_index(level=0, inplace=True)
results.sort_values(by="score", ascending=False, inplace=True, ignore_index=True)



In [281]:
for i in range(len(results)):
    if results.loc[i, "score"] > 0.001:
        print(str(labels[results.loc[i, "index"]]) + " (" + str(round(100 * results.loc[i, "score"])) + "%)")



Watermelon (33%)
Carrot (33%)
Potato (33%)


In [235]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.model_selection import train_test_split

In [236]:
from keras.utils import to_categorical
df = pd.read_csv(r"npk_new.csv")
print(df)

le = LabelEncoder()
le.fit(df['LABEL'])
df['LABEL']=le.transform(df['LABEL'])


scaler = StandardScaler()
#scaler.fit(X)
df.iloc[0:,:-1] =scaler.fit_transform(df.iloc[0:,:-1]) 

X=df.drop('LABEL',axis=1)
Y=df['LABEL']
print(X.shape)
print(Y.shape)

       N   P   K       LABEL
0     80  77  73       Beans
1     63  73  74       Beans
2     85  61  78       Beans
3    116  59  54       Beans
4     96  64  69       Beans
..   ...  ..  ..         ...
420  118  47  77  Watermelon
421  157  57  66  Watermelon
422  144  43  77  Watermelon
423  148  47  61  Watermelon
424  109  58  61  Watermelon

[425 rows x 4 columns]
(425, 3)
(425,)


In [237]:

print(X.shape)
print(Y.shape)
print(df)

(425, 3)
(425,)
            N         P         K  LABEL
0   -1.217038  1.067362  0.289705      0
1   -1.632572  0.739477  0.353467      0
2   -1.094823 -0.244178  0.608516      0
3   -0.337085 -0.408120 -0.921775      0
4   -0.825948  0.001736  0.034657      0
..        ...       ...       ...    ...
420 -0.288199 -1.391775  0.544753     16
421  0.665083 -0.572063 -0.156630     16
422  0.347323 -1.719660  0.544753     16
423  0.445095 -1.391775 -0.475440     16
424 -0.508187 -0.490091 -0.475440     16

[425 rows x 4 columns]


In [238]:

nb_class=17

X = np.expand_dims(X,axis=1)
Y = to_categorical(Y, nb_class)
print(X.shape)
print(Y.shape)

Y

X

xtrain, xtest, ytrain, ytest = train_test_split(X,Y, test_size=0.2, random_state=100)

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)



nb_timesteps=X.shape[2]
nb_timesteps


(425, 1, 3)
(425, 17)
(340, 1, 3)
(85, 1, 3)
(340, 17)
(85, 17)


3

In [239]:

#LSTM
epochs=150
model = Sequential()
model.add(LSTM(200, input_shape=(1, nb_timesteps)))
model.add(Dense(nb_class,activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
#history = model.fit(trainX, trainY, epochs=150, batch_size=100, validation_data=(testX, testY), verbose=0, shuffle=False)

history=model.fit(xtrain, ytrain, epochs=epochs, batch_size=30, validation_data=(xtest, ytest))


Epoch 1/150
12/12 [==============================] - 7s 154ms/step - loss: 2.8103 - accuracy: 0.1294 - val_loss: 2.7964 - val_accuracy: 0.1529
Epoch 2/150
12/12 [==============================] - 0s 10ms/step - loss: 2.7604 - accuracy: 0.2294 - val_loss: 2.7536 - val_accuracy: 0.1882
Epoch 3/150
12/12 [==============================] - 0s 13ms/step - loss: 2.7084 - accuracy: 0.2559 - val_loss: 2.7077 - val_accuracy: 0.1882
Epoch 4/150
12/12 [==============================] - 0s 10ms/step - loss: 2.6519 - accuracy: 0.2559 - val_loss: 2.6550 - val_accuracy: 0.1882
Epoch 5/150
12/12 [==============================] - 0s 13ms/step - loss: 2.5868 - accuracy: 0.2500 - val_loss: 2.5970 - val_accuracy: 0.1882
Epoch 6/150
12/12 [==============================] - 0s 10ms/step - loss: 2.5180 - accuracy: 0.2529 - val_loss: 2.5332 - val_accuracy: 0.2000
Epoch 7/150
12/12 [==============================] - 0s 10ms/step - loss: 2.4423 - accuracy: 0.2500 - val_loss: 2.4713 - val_accuracy: 0.2000
Epoch

12/12 [==============================] - 0s 9ms/step - loss: 1.4990 - accuracy: 0.4676 - val_loss: 1.6691 - val_accuracy: 0.3412
Epoch 59/150
12/12 [==============================] - 0s 10ms/step - loss: 1.4955 - accuracy: 0.4706 - val_loss: 1.6671 - val_accuracy: 0.3412
Epoch 60/150
12/12 [==============================] - 0s 10ms/step - loss: 1.4929 - accuracy: 0.4706 - val_loss: 1.6622 - val_accuracy: 0.3412
Epoch 61/150
12/12 [==============================] - 0s 11ms/step - loss: 1.4877 - accuracy: 0.4676 - val_loss: 1.6592 - val_accuracy: 0.3412
Epoch 62/150
12/12 [==============================] - 0s 10ms/step - loss: 1.4866 - accuracy: 0.4794 - val_loss: 1.6530 - val_accuracy: 0.3412
Epoch 63/150
12/12 [==============================] - 0s 11ms/step - loss: 1.4825 - accuracy: 0.4882 - val_loss: 1.6476 - val_accuracy: 0.3529
Epoch 64/150
12/12 [==============================] - 0s 9ms/step - loss: 1.4787 - accuracy: 0.4824 - val_loss: 1.6475 - val_accuracy: 0.3529
Epoch 65/150
1

12/12 [==============================] - 0s 6ms/step - loss: 1.3798 - accuracy: 0.5118 - val_loss: 1.5765 - val_accuracy: 0.3882
Epoch 116/150
12/12 [==============================] - 0s 7ms/step - loss: 1.3793 - accuracy: 0.5088 - val_loss: 1.5738 - val_accuracy: 0.3647
Epoch 117/150
12/12 [==============================] - 0s 7ms/step - loss: 1.3765 - accuracy: 0.5059 - val_loss: 1.5753 - val_accuracy: 0.3765
Epoch 118/150
12/12 [==============================] - 0s 6ms/step - loss: 1.3760 - accuracy: 0.5059 - val_loss: 1.5747 - val_accuracy: 0.3647
Epoch 119/150
12/12 [==============================] - 0s 7ms/step - loss: 1.3743 - accuracy: 0.5059 - val_loss: 1.5722 - val_accuracy: 0.3647
Epoch 120/150
12/12 [==============================] - 0s 7ms/step - loss: 1.3735 - accuracy: 0.5088 - val_loss: 1.5708 - val_accuracy: 0.3765
Epoch 121/150
12/12 [==============================] - 0s 7ms/step - loss: 1.3726 - accuracy: 0.5118 - val_loss: 1.5688 - val_accuracy: 0.3765
Epoch 122/150

In [240]:
import tensorflow
model.save('lstm_model.h5')
model =tensorflow.keras.models.load_model('lstm_model.h5')

In [241]:
result=['Beans',
 'Brinjal',
 'Cabbage',
 'Carrot',
 'Cauliflower',
 'Chilli',
 'Cowpea',
 'Cucumber',
 'Garden pea',
 'Garlic',
 'Lettuce',
 'Okra',
 'Onion',
 'Potato',
 'Snap bean',
 'Spinach',
 'Tomato',
 'Watermelon']

In [245]:
import tensorflow as tf
test_data = np.array([[0,0,90]])
lstm_model =tf.keras.models.load_model('lstm_model.h5')
o=lstm_model.predict(test_data.reshape(-1,1,3), batch_size=1)

print(o)
print(len(o[0]))
print(o.argmax())
print(result[int(o.argmax())])

1/1 [==============================] - 1s 1s/step
[[1.6474480e-13 1.4375312e-04 2.8041068e-11 1.2967738e-03 1.3796043e-05
  1.7659418e-12 3.3267689e-13 6.5825343e-14 2.6332276e-09 3.6484052e-10
  1.2283412e-11 7.6126473e-11 9.9849522e-01 7.5307295e-12 6.3873620e-15
  5.0425340e-05 2.1601143e-12]]
17
12
Onion
